# Import

In [1]:
import pandas as pd
import numpy as np

# from IPython.display import display
# pd.options.display.max_columns = 500
# pd.options.display.max_rows = 500

%matplotlib inline

In [2]:
train_raw = pd.read_csv('data/train.csv')

In [3]:
train_df = pd.read_csv('data/train.csv')
train_df = train_df.drop('Id', axis = 1)

In [4]:
test_df = pd.read_csv('data/test.csv')

In [5]:
y = train_df['SalePrice']
train_df = train_df.drop('SalePrice', axis=1)

# Divide varbles by type

In [6]:
# date variables
date_vars = ['YearBuilt', 'YearRemodAdd', 'GarageYrBlt', 'MoSold', 'YrSold']

# list comprehension of continuous variables that are not dates
cont_vars = [cont_var for cont_var in train_df.columns if
        (train_df[cont_var].dtype == 'float64') | (train_df[cont_var].dtype == 'int64')
        and not cont_var in date_vars]

# change continus varibles to float
train_df[cont_vars] = train_df[cont_vars].astype('float64')

# create dataframe of date variables
dates = train_df[date_vars].astype(object)

In [7]:
# dict compehension of the frequncy of unique varibales
cont_freq = {cont_var: train_df[cont_var].nunique() for cont_var in cont_vars}
# frequency data frame
cont_freq_df = pd.DataFrame.from_dict(cont_freq, orient='index').rename(columns={0: 'freq'})
# continuous variables with few unique values
low_vol_cont = ['LowQualFinSF', '3SsnPorch', 'PoolArea']
# variabkes with Nan as the mode
mode_na = ['Alley', 'PoolQC', 'Fence','MiscFeature', 'FireplaceQu']


In [8]:
# data frame with numerical variables 
nums = train_df[list(cont_freq_df[cont_freq_df.freq >= 50].index)].astype(object)

num_cats = train_df[list(cont_freq_df[cont_freq_df.freq < 50].index)].astype(object)

# list of non categorical variables
non_cat = list(nums.columns) + list(dates.columns) + list(num_cats.columns) + mode_na
# data frame with only categorical
cats =  train_df.drop(non_cat, axis = 1)

# data frame with categoricals where the mode is nan
cats_na = train_df[mode_na]

# move continuous variables with low-frequency from num_cats to nums
nums[low_vol_cont] = num_cats[low_vol_cont]
num_cats = num_cats.drop(low_vol_cont, axis=1)

# Fill numarical missing with 0

In [9]:
nums = nums.fillna(0)

# Add dummies remove dominate and original categorical columns

In [10]:
def add_dummies_remove_modes(dummy_list, df_raw, df):
    """ 
    add_dummies_remove_modes (dummy_list, df_raw, df)
    takes a list of column names `dummy_list` to dummify then drop 
    after dummification, a reference dataframe `df_raw` to search for 
    the dominant value of each varable in `dummy_list` then drop the 
    dominant dummy variable and a dataframe `df` you wish to concat 
    dummified variables to.
    """

    dummy_modes = list(df_raw[dummy_list].mode().iloc[0,:].items())

    dummy_modes = [(col, (float(mode))) 
                   if type(mode) == int else (col, mode) for col, mode in dummy_modes]

    drop_modes = list(map(lambda x: str(x[0]) + '_' + str(x[1]), dummy_modes))

    dummy_cols = pd.get_dummies(
        df[dummy_list].astype(object), dummy_na=True).drop(drop_modes, axis=1)

    return pd.concat([
       df.drop(dummy_list, axis=1),
       dummy_cols], axis=1)


# Numarical Categorical to Dummies

In [11]:
num_cats_raw = num_cats

# impute numarical categoricals with mode
num_cats = num_cats.fillna(num_cats.mode())

# dummify categotical numericals
num_cats = add_dummies_remove_modes(list(num_cats.columns), train_df, num_cats)

# drop columns that are all 0
num_cats = num_cats.drop(
    num_cats.columns[num_cats.sum()==0], axis=1)


# Categorical to Dummies

In [12]:
cats_raw = cats

#cats dummies
cats = add_dummies_remove_modes(list(cats.columns), train_df, cats)

cats = cats.drop(cats.columns[cats.sum()==0], axis=1)

# Dummify columns with NA as mode

In [13]:
cats_na_raw = cats_na
cats_na = pd.get_dummies(cats_na)

cats_na = cats_na.drop(cats_na.columns[cats_na.sum()==0], axis=1)

# Change GarageYrBlt to binary variable

In [14]:
dates['GarageYrBlt'] = dates['GarageYrBlt'].isna().apply(lambda x: int(not x))

In [15]:
test_df['GarageYrBlt'] = test_df['GarageYrBlt'].isna().apply(lambda x: int(not x))

# Concat final data frame

In [16]:
df_train_final = pd.concat([dates, nums, num_cats, cats, cats_na], axis=1)

# Quality Check

In [17]:
df_train_final.shape

(1460, 337)

In [18]:
cols_before_drop = set(df_train_final.columns)
cols_after_drop = set(df_train_final.T.drop_duplicates(keep='first').T.columns)
cols_before_drop - cols_after_drop

{'BldgType_Duplex',
 'BsmtCond_nan',
 'BsmtFinType1_nan',
 'Condition2_RRAe',
 'Exterior2nd_CBlock',
 'GarageCond_nan',
 'GarageFinish_nan',
 'GarageQual_nan',
 'GarageType_nan',
 'TotRmsAbvGrd_14.0'}

In [19]:
cols_after_drop - cols_before_drop

set()

In [20]:
df_train_final = df_train_final.T.drop_duplicates(keep='first').T

In [21]:
df_train_final.shape

(1460, 327)

# Transform Dummies on Test

In [22]:
test_dummy_cols = num_cats_raw.columns.tolist() +  cats_raw.columns.tolist() +  cats_na_raw.columns.tolist()

In [23]:
 test_dummies1 = pd.get_dummies(test_df[test_dummy_cols])

In [24]:
test_df = pd.concat([test_dummies1, test_df], axis=1)

In [25]:
train_cols = set(df_train_final.columns)
test_cols = set(test_df.columns)

test_drop =  list(test_cols - train_cols)
test_unkonwn_dummies = list(train_cols - test_cols)

# Quality Check on Test

In [26]:
# these are all either the varibles that need to be droped after dummifyin 
# or are variables show up in train but not test
test_drop

['BsmtFinType1',
 'MSZoning',
 'KitchenQual',
 'Exterior1st',
 'GarageType',
 'BldgType_Duplex',
 'GarageCars',
 'Condition1',
 'LotShape',
 'GarageCond_TA',
 'GarageQual',
 'BedroomAbvGr',
 'FullBath',
 'CentralAir_Y',
 'Alley',
 'ExterCond',
 'Utilities_AllPub',
 'BsmtFullBath',
 'Neighborhood',
 'RoofMatl',
 'Exterior2nd_VinylSd',
 'HouseStyle_1Story',
 'MiscFeature',
 'Exterior2nd_CBlock',
 'Street',
 'Neighborhood_NAmes',
 'RoofStyle_Gable',
 'BldgType_1Fam',
 'KitchenQual_TA',
 'OverallQual',
 'Condition1_Norm',
 'Utilities',
 'LandContour_Lvl',
 'RoofMatl_CompShg',
 'PavedDrive_Y',
 'GarageCond',
 'Functional_Typ',
 'BsmtQual',
 'BsmtExposure_No',
 'Fence',
 'BsmtQual_TA',
 'SaleType',
 'HeatingQC_Ex',
 'BsmtCond',
 'Exterior1st_VinylSd',
 'BsmtFinType2',
 'OverallCond',
 'BldgType',
 'Foundation',
 'BsmtFinType1_Unf',
 'RoofStyle',
 'Foundation_PConc',
 'LotShape_Reg',
 'SaleType_WD',
 'Electrical',
 'HalfBath',
 'CentralAir',
 'BsmtFinType2_Unf',
 'KitchenAbvGr',
 'HeatingQC',

In [27]:
# these are the dummy variables that do not show up in test
test_unkonwn_dummies

['MiscVal_500.0',
 'OverallQual_2.0',
 'OverallQual_4.0',
 'MiscVal_450.0',
 'TotRmsAbvGrd_11.0',
 'GarageCars_4.0',
 'GarageCars_3.0',
 'BsmtExposure_nan',
 'OverallQual_8.0',
 'Exterior1st_ImStucc',
 'OverallCond_3.0',
 'MSSubClass_160.0',
 'MiscVal_1400.0',
 'OverallQual_7.0',
 'OverallCond_6.0',
 'KitchenAbvGr_2.0',
 'OverallQual_10.0',
 'MiscVal_800.0',
 'Fireplaces_2.0',
 'BedroomAbvGr_8.0',
 'TotRmsAbvGrd_8.0',
 'MasVnrType_nan',
 'BsmtHalfBath_1.0',
 'OverallCond_1.0',
 'TotRmsAbvGrd_12.0',
 'BsmtFullBath_1.0',
 'MSSubClass_85.0',
 'MiscVal_700.0',
 'MiscVal_15500.0',
 'BedroomAbvGr_5.0',
 'MSSubClass_90.0',
 'MiscFeature_TenC',
 'MSSubClass_40.0',
 'MSSubClass_30.0',
 'Heating_OthW',
 'BedroomAbvGr_2.0',
 'OverallCond_4.0',
 'MiscVal_480.0',
 'Exterior1st_Stone',
 'BsmtHalfBath_2.0',
 'HalfBath_1.0',
 'MiscVal_1150.0',
 'FullBath_3.0',
 'TotRmsAbvGrd_7.0',
 'TotRmsAbvGrd_10.0',
 'BsmtFinType2_nan',
 'MiscVal_560.0',
 'OverallCond_8.0',
 'MiscVal_8300.0',
 'TotRmsAbvGrd_5.0',
 

# Drop test columns that are not in train

In [28]:
test_df = test_df.drop(test_drop, axis=1)

# Add dummies that are unseen in test

In [29]:
test_dummies2 = test_df.reindex(columns = test_unkonwn_dummies, fill_value=0)

In [30]:
test_df = pd.concat([test_df, test_dummies2], axis=1)

# Impute missing with 0

In [31]:
test_na_cols = test_df.columns[test_df.isna().sum() > 0]
test_df[test_na_cols] = test_df[test_na_cols].fillna(0)

# Matching order of train and test

In [32]:
test_df = test_df[df_train_final.columns]

# Add response variable

In [33]:
df_train_final['SalePrice'] = train_raw['SalePrice']

# Pickle

In [34]:
df_train_final.to_pickle('train.pkl')
test_df.to_pickle('test.pkl')